# 酒店是否好评的二分类任务

In [1]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

d:\workplaces\miniconda_py39\envs\hugging\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#导入数据
import pandas as pd
data = pd.read_csv('./ChnSentiCorp_htl_all.csv',sep=',')
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [3]:
#创建Dataset
from torch.utils.data import Dataset
class MyDataSet(Dataset):
    def __init__(self,data):
        super().__init__()
        self.data = data 
    def __getitem__(self, index):
        label = self.data.iloc[index]['label']
        review = self.data.iloc[index]['review']
        return review,label
    def __len__(self):
        return len(self.data)
dataset = MyDataSet(data)
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


In [4]:
#拆分训练集和测试集
from torch.utils.data import random_split
train_dataset,valid_dataset = random_split(dataset,lengths=[0.8,0.2])
len(train_dataset),len(valid_dataset)

(6212, 1553)

In [5]:
#collate_fn并不是写在dataset里面的，因为这样一次一次地调用会非常浪费时间，所以写在Dataloader创建那里
import torch
tokenizer = AutoTokenizer.from_pretrained('./rbt3')
def collate_fn(batch):
    labels,reviews = [], []
    for item in batch:
        reviews.append(item[0])
        labels.append(item[1])
    
    inputs = tokenizer(reviews,max_length=128,padding="max_length",truncation=True,return_tensors='pt')#注意input的结果其实是一个字典，里面有一个键值对专门放labels
    inputs['labels'] = torch.tensor(labels)

    return inputs

In [6]:
#创建dataloader返回batch数据
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True,collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset,batch_size=32,shuffle=True,collate_fn=collate_fn)

In [7]:
for k,v in next(enumerate(train_loader))[1].items():
    print(k,v.shape)

input_ids torch.Size([32, 128])
token_type_ids torch.Size([32, 128])
attention_mask torch.Size([32, 128])
labels torch.Size([32])


In [8]:
#创建模型和优化器
model = AutoModelForSequenceClassification.from_pretrained('./rbt3')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from torch.optim  import Adam
optimizer = Adam(model.parameters(),lr=2e-5)

In [10]:
#开始训练函数和测试函数
if torch.cuda.is_available():
    model = model.cuda()

#标准代码

def evaluate():
    model.eval()
    acc_nums = 0
    with torch.inference_mode():
        for batch in valid_loader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits,dim=-1)
            acc_nums += (pred.long() == batch['lables'].long()).float().sum() 
            
    return acc_nums / len(valid_dataset)
def train(epoch=2,log_step=100):
    global_step =0
    for ep in range(epoch):
        model.train()
        for batch in train_loader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            global_step += 1
            if global_step % log_step == 0:
                print(f"ep:{ep},global_step:{global_step},loss:{output.loss.item()}")
        acc = evaluate()
        print(f'ep:{ep},acc:{acc}')

In [25]:
train()

KeyboardInterrupt: 

In [11]:
#模型预测
sen = '这家餐馆很好，价格便宜，并且饭量很多'
id2_labels = {0:'差评',1:'好评'}
model.eval()
with torch.inference_mode():
    input = tokenizer(sen,return_tensors='pt')
    output = model(**input)
    print( torch.argmax(output.logits,dim=-1) )

tensor([1])


In [14]:
model.config.id2label = id2_labels
model.config.id2label

{0: '差评', 1: '好评'}

In [15]:
#一步到位 pipeline
from transformers import pipeline
sen = '这家餐馆很垃圾，价格贵，并且饭量很少，cnm，lsw，sb，sb，sb，sb，nt，我受不了'
#model.config.id2_label = id2_labels
pipe = pipeline('text-classification',model = model ,tokenizer=tokenizer,device='cpu')
pipe(sen)

[{'label': '好评', 'score': 0.6180132627487183}]